# 📈 Multi-Agent Investment Research Platform

Welcome to the Multi-Agent Investment Research Platform. This notebook uses a team of specialized AI agents to perform a comprehensive analysis of a stock ticker. Each agent has a specific role, from data collection to fundamental, technical, and sentiment analysis, culminating in a final recommendation from a portfolio manager agent.

**Key Components:**
1.  **Setup & Imports**: Configure the environment and import necessary libraries.
2.  **Data Structures**: Define the standard formats for agent results and shared context.
3.  **Shared Tools**: A toolkit of functions for fetching financial data, shared across all agents.
4.  **Agent Definitions**: Implementation of each specialized agent.
5.  **Orchestrator**: The main platform that manages the workflow between agents.
6.  **Command Center**: An interactive interface to run different types of analyses.

---

## Cell 1: Setup and Imports

This cell handles all the necessary setup, including:
- Importing Python libraries (`os`, `asyncio`, `pandas`, `yfinance`, etc.).
- Applying `nest_asyncio` to allow asynchronous operations within the Jupyter environment.
- **Setting API keys**. You must add your `GOOGLE_API_KEY` to proceed.

In [1]:
pip install yfinance 


[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import nest_asyncio
nest_asyncio.apply()

import os
import asyncio
import json
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Any, Tuple
import pandas as pd
import numpy as np
from dataclasses import dataclass, asdict
from enum import Enum

# IMPORTANT: Set your API keys here
# You can get a Google Gemini API key from Google AI Studio.
google_key = os.environ.get("GOOGLE_API_KEY")
sec_key = os.environ.get("SEC_API_KEY")
news_key = os.environ.get("NEWS_API_KEY")

print(f"Google API Key loaded: {'Yes' if google_key else 'No. Check your .zshrc file and restart the terminal.'}")
print(f"SEC API Key loaded: {'Yes' if sec_key else 'No. Check your .zshrc file and restart the terminal.'}")
print(f"NEWS API Key loaded: {'Yes' if news_key else 'No. Check your .zshrc file and restart the terminal.'}")
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"



# Google imports
import google.generativeai as genai

# Configure Gemini
genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

# External imports
import yfinance as yf
import warnings
warnings.filterwarnings("ignore")

print("✅ Setup complete!")

Google API Key loaded: Yes
SEC API Key loaded: Yes
NEWS API Key loaded: Yes
✅ Setup complete!


/Users/zc/Desktop/agent/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Cell 2: Data Structures

Here, we define the standardized data classes that ensure consistent data flow and communication between agents.

- `AnalysisStatus`: An enumeration for tracking the state of an analysis.
- `AgentResult`: A dataclass to hold the output from any agent, ensuring a uniform structure.
- `InvestmentContext`: A shared object that is passed between agents, carrying key information like the ticker, company info, and the results from other agents.

In [3]:
class AnalysisStatus(Enum):
    PENDING = "pending"
    IN_PROGRESS = "in_progress"
    COMPLETED = "completed"
    FAILED = "failed"

@dataclass
class AgentResult:
    """Standardized result format for all agents."""
    agent_name: str
    ticker: str
    status: AnalysisStatus
    confidence_score: float  # 0-1
    recommendation: str  # BUY/HOLD/SELL
    key_metrics: Dict[str, Any]
    analysis_text: str
    risks: List[str]
    opportunities: List[str]
    timestamp: str

@dataclass
class InvestmentContext:
    """Shared context passed between agents."""
    ticker: str
    company_name: str
    sector: str
    market_cap: float
    current_price: float
    session_id: str
    analysis_results: Dict[str, AgentResult]
    
    def add_result(self, result: AgentResult):
        self.analysis_results[result.agent_name] = result
        
print("✅ Data structures defined.")

✅ Data structures defined.


## Cell 3: Shared Tools

This class, `SharedInvestmentTools`, contains all the functions needed to fetch data from external sources like Yahoo Finance (`yfinance`). These tools are instantiated once and passed to any agent that needs them, promoting code reuse and efficiency. It includes functions for:
- Basic company information
- Financial statements
- Historical market data
- Technical indicator calculations
- Simulated news sentiment analysis

In [4]:
class SharedInvestmentTools:
    """Comprehensive tools shared across all agents."""
    
    def __init__(self):
        self._cache = {}
    
    def get_basic_company_info(self, ticker: str) -> str:
        """Quick company lookup."""
        cache_key = f"basic_info_{ticker}"
        if cache_key in self._cache:
            return self._cache[cache_key]
        
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            basic_info = {
                "name": info.get('longName', 'Unknown'),
                "sector": info.get('sector', 'Unknown'),
                "industry": info.get('industry', 'Unknown'),
                "market_cap": info.get('marketCap', 0),
                "current_price": info.get('currentPrice', info.get('previousClose', 0))
            }
            
            result = json.dumps(basic_info, indent=2)
            self._cache[cache_key] = result
            return result
            
        except Exception as e:
            print(f"Error getting basic info for {ticker}: {e}")
            raise
    
    def fetch_financial_statements(self, ticker: str) -> str:
        """Fetch financial data."""
        try:
            stock = yf.Ticker(ticker)
            info = stock.info
            
            # Get key financial metrics
            revenue = info.get('totalRevenue', 0)
            net_income = info.get('netIncomeToCommon', 0)
            profit_margin = info.get('profitMargins', 0)
            
            result = f"""Financial Data for {ticker}:
- Revenue: ${revenue:,.0f}
- Net Income: ${net_income:,.0f}
- Profit Margin: {profit_margin:.2%}
- P/E Ratio: {info.get('trailingPE', 'N/A')}
- P/B Ratio: {info.get('priceToBook', 'N/A')}
- Debt/Equity: {info.get('debtToEquity', 'N/A')}
"""
            return result
            
        except Exception as e:
            return f"Error fetching financial statements: {str(e)}"
    
    def get_market_data(self, ticker: str) -> str:
        """Get market data and price history."""
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="1y")
            info = stock.info
            
            if hist.empty:
                return "No market data available."
            
            current_price = hist['Close'].iloc[-1]
            high_52w = hist['High'].max()
            low_52w = hist['Low'].min()
            price_change_1y = (current_price / hist['Close'].iloc[0] - 1) * 100
            
            result = f"""Market Data for {ticker}:
- Current Price: ${current_price:.2f}
- 52-Week High: ${high_52w:.2f}
- 52-Week Low: ${low_52w:.2f}
- 1-Year Change: {price_change_1y:+.2f}%
- Volume (Avg): {hist['Volume'].mean():,.0f}
- Beta: {info.get('beta', 'N/A')}
"""
            return result
            
        except Exception as e:
            return f"Error fetching market data: {str(e)}"
    
    def calculate_technical_indicators(self, ticker: str) -> str:
        """Calculate technical indicators."""
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(period="3mo")
            
            if hist.empty:
                return "No data for technical analysis."
            
            # Simple moving averages
            hist['MA20'] = hist['Close'].rolling(20).mean()
            hist['MA50'] = hist['Close'].rolling(50).mean()
            
            # RSI
            delta = hist['Close'].diff()
            gain = (delta.where(delta > 0, 0)).rolling(14).mean()
            loss = (-delta.where(delta < 0, 0)).rolling(14).mean()
            rs = gain / loss
            hist['RSI'] = 100 - (100 / (1 + rs))
            
            current_price = hist['Close'].iloc[-1]
            current_rsi = hist['RSI'].iloc[-1]
            ma20 = hist['MA20'].iloc[-1]
            ma50 = hist['MA50'].iloc[-1] if not pd.isna(hist['MA50'].iloc[-1]) else ma20
            
            # Determine signals
            trend_signal = "Bullish" if current_price > ma20 > ma50 else "Bearish"
            rsi_signal = "Overbought" if current_rsi > 70 else "Oversold" if current_rsi < 30 else "Neutral"
            
            result = f"""Technical Indicators for {ticker}:
- Price: ${current_price:.2f}
- 20-day MA: ${ma20:.2f} {'✅' if current_price > ma20 else '❌'}
- 50-day MA: ${ma50:.2f} {'✅' if current_price > ma50 else '❌'}
- RSI (14): {current_rsi:.1f} ({rsi_signal})
- Trend: {trend_signal}
"""
            return result
            
        except Exception as e:
            return f"Error calculating technical indicators: {str(e)}"
    
    def analyze_sentiment(self, ticker: str) -> str:
        """Simulate news sentiment analysis."""
        # In production, this would use NEWS_API_KEY
        # For demo, we'll simulate
        import random
        
        sentiment_score = random.uniform(-1, 1)
        num_articles = random.randint(5, 20)
        
        if sentiment_score > 0.3:
            sentiment = "Positive"
        elif sentiment_score < -0.3:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
        
        result = f"""News Sentiment for {ticker}:
- Overall Sentiment: {sentiment} ({sentiment_score:.2f})
- Articles Analyzed: {num_articles}
- Recent News: Simulated (News API not configured)
"""
        return result
        
print("✅ Shared tools defined.")

✅ Shared tools defined.


## Cell 4: Agent Implementations

This section contains the core logic of the platform: the specialized agents. Each agent is a class with a specific `analyze` method. They use the `SharedInvestmentTools` to get data and then prompt the Gemini model to perform their specialized analysis.

1.  **DataCollectionAgent**: Gathers all necessary raw data and performs a quality check.
2.  **FundamentalAnalysisAgent**: Analyzes the financial health and valuation of the company.
3.  **TechnicalAnalysisAgent**: Analyzes price trends, chart patterns, and technical indicators.
4.  **SentimentAnalysisAgent**: Gauges market sentiment based on news and social media (simulated).
5.  **PortfolioManagerAgent**: The final agent that synthesizes all prior analyses, weighs their recommendations, and provides a final, unified investment thesis.

In [5]:
class DataCollectionAgent:
    """Agent 1: Responsible for gathering all raw data."""
    
    def __init__(self, tools: SharedInvestmentTools):
        self.tools = tools
        self.model = genai.GenerativeModel('gemini-2.0-flash')
    
    async def collect_data(self, ticker: str) -> AgentResult:
        """Execute data collection."""
        
        # Gather all data
        basic_info = self.tools.get_basic_company_info(ticker)
        financial_data = self.tools.fetch_financial_statements(ticker)
        market_data = self.tools.get_market_data(ticker)
        
        # Combine data
        all_data = f"""
Data Collection for {ticker}:

{basic_info}

{financial_data}

{market_data}
"""
        
        # Use Gemini to analyze data quality
        prompt = f"""
As a data analyst, assess the quality and completeness of this financial data:

{all_data}

Provide:
1. Data quality score (0-1)
2. Any missing critical information
3. Overall assessment
"""
        
        try:
            response = self.model.generate_content(prompt)
            analysis_text = response.text
        except Exception as e:
            analysis_text = f"Data collected. Analysis: {str(e)}"
        
        return AgentResult(
            agent_name="data_collector",
            ticker=ticker,
            status=AnalysisStatus.COMPLETED,
            confidence_score=0.85,
            recommendation="DATA_COLLECTED",
            key_metrics={"data_quality": 0.85},
            analysis_text=all_data + "\n\n" + analysis_text,
            risks=[],
            opportunities=["Complete dataset available"],
            timestamp=datetime.now().isoformat()
        )

class FundamentalAnalysisAgent:
    """Agent 2: Fundamental analysis."""
    
    def __init__(self, tools: SharedInvestmentTools):
        self.tools = tools
        self.model = genai.GenerativeModel('gemini-gemini-2.0-flash')
    
    async def analyze_fundamentals(self, context: InvestmentContext) -> AgentResult:
        """Perform fundamental analysis."""
        
        financial_data = self.tools.fetch_financial_statements(context.ticker)
        
        prompt = f"""
As a fundamental analyst, analyze {context.ticker} ({context.company_name}):

Company: {context.company_name}
Sector: {context.sector}
Market Cap: ${context.market_cap:,.0f}
Current Price: ${context.current_price:.2f}

Financial Data:
{financial_data}

Provide:
1. Financial health assessment
2. Valuation analysis (undervalued/fair/overvalued)
3. BUY/HOLD/SELL recommendation
4. Confidence level (0-1)
5. Key risks and opportunities
"""
        
        try:
            response = self.model.generate_content(prompt)
            analysis_text = response.text
            
            # Extract recommendation
            if "BUY" in analysis_text.upper():
                recommendation = "BUY"
            elif "SELL" in analysis_text.upper():
                recommendation = "SELL"
            else:
                recommendation = "HOLD"
                
        except Exception as e:
            analysis_text = f"Analysis error: {str(e)}"
            recommendation = "HOLD"
        
        return AgentResult(
            agent_name="fundamental_analyst",
            ticker=context.ticker,
            status=AnalysisStatus.COMPLETED,
            confidence_score=0.75,
            recommendation=recommendation,
            key_metrics={"valuation": "completed"},
            analysis_text=analysis_text,
            risks=["Valuation risk", "Market risk"],
            opportunities=["Growth potential", "Strong fundamentals"],
            timestamp=datetime.now().isoformat()
        )

class TechnicalAnalysisAgent:
    """Agent 3: Technical analysis."""
    
    def __init__(self, tools: SharedInvestmentTools):
        self.tools = tools
        self.model = genai.GenerativeModel('gemini-2.0-flash')

    async def analyze_technicals(self, context: InvestmentContext) -> AgentResult:
        """Perform technical analysis."""
        
        market_data = self.tools.get_market_data(context.ticker)
        technical_data = self.tools.calculate_technical_indicators(context.ticker)
        
        prompt = f"""
As a technical analyst, analyze {context.ticker}:

{market_data}

{technical_data}

Provide:
1. Trend analysis
2. Support and resistance levels
3. BUY/HOLD/SELL recommendation based on technicals
4. Entry/exit points
5. Confidence level (0-1)
"""
        
        try:
            response = self.model.generate_content(prompt)
            analysis_text = response.text
            
            # Extract recommendation
            if "BUY" in analysis_text.upper() or "BULLISH" in analysis_text.upper():
                recommendation = "BUY"
            elif "SELL" in analysis_text.upper() or "BEARISH" in analysis_text.upper():
                recommendation = "SELL"
            else:
                recommendation = "HOLD"
                
        except Exception as e:
            analysis_text = f"Technical analysis: {technical_data}"
            recommendation = "HOLD"
        
        return AgentResult(
            agent_name="technical_analyst",
            ticker=context.ticker,
            status=AnalysisStatus.COMPLETED,
            confidence_score=0.70,
            recommendation=recommendation,
            key_metrics={"trend": "analyzed"},
            analysis_text=analysis_text,
            risks=["Technical breakdown risk"],
            opportunities=["Momentum opportunity"],
            timestamp=datetime.now().isoformat()
        )

class SentimentAnalysisAgent:
    """Agent 4: Sentiment analysis."""
    
    def __init__(self, tools: SharedInvestmentTools):
        self.tools = tools
        self.model = genai.GenerativeModel('gemini-2.0-flash')

    async def analyze_sentiment(self, context: InvestmentContext) -> AgentResult:
        """Analyze market sentiment."""
        
        sentiment_data = self.tools.analyze_sentiment(context.ticker)
        
        prompt = f"""
As a sentiment analyst, analyze market sentiment for {context.ticker}:

{sentiment_data}

Company: {context.company_name}
Sector: {context.sector}

Provide:
1. Overall market sentiment assessment
2. Impact on stock price
3. BUY/HOLD/SELL recommendation based on sentiment
4. Confidence level (0-1)
"""
        
        try:
            response = self.model.generate_content(prompt)
            analysis_text = response.text
            
            if "POSITIVE" in analysis_text.upper() and "BUY" in analysis_text.upper():
                recommendation = "BUY"
            elif "NEGATIVE" in analysis_text.upper() and "SELL" in analysis_text.upper():
                recommendation = "SELL"
            else:
                recommendation = "HOLD"
                
        except Exception as e:
            analysis_text = sentiment_data
            recommendation = "HOLD"
        
        return AgentResult(
            agent_name="sentiment_analyst",
            ticker=context.ticker,
            status=AnalysisStatus.COMPLETED,
            confidence_score=0.65,
            recommendation=recommendation,
            key_metrics={"sentiment": "analyzed"},
            analysis_text=analysis_text,
            risks=["Sentiment reversal risk"],
            opportunities=["Positive momentum"],
            timestamp=datetime.now().isoformat()
        )

class PortfolioManagerAgent:
    """Agent 5: Portfolio synthesis and final recommendation."""
    
    def __init__(self):
        self.model = genai.GenerativeModel('gemini-2.0-flash')

    async def synthesize_analysis(self, context: InvestmentContext) -> AgentResult:
        """Synthesize all analyses."""
        
        # Compile all analyses
        all_analyses = ""
        buy_votes = 0
        hold_votes = 0
        sell_votes = 0
        total_confidence = 0
        
        for agent_name, result in context.analysis_results.items():
            all_analyses += f"\n{agent_name}: {result.recommendation} (Confidence: {result.confidence_score:.1%})\n"
            all_analyses += f"{result.analysis_text[:300]}...\n\n"
            
            if result.recommendation == "BUY":
                buy_votes += result.confidence_score
            elif result.recommendation == "SELL":
                sell_votes += result.confidence_score
            else:
                hold_votes += result.confidence_score
            
            total_confidence += result.confidence_score
        
        avg_confidence = total_confidence / len(context.analysis_results) if context.analysis_results else 0
        
        prompt = f"""
As a senior portfolio manager, synthesize these analyses for {context.ticker}:

{all_analyses}

Voting Summary:
- Buy signals: {buy_votes:.2f}
- Hold signals: {hold_votes:.2f}
- Sell signals: {sell_votes:.2f}

Provide:
1. Final BUY/HOLD/SELL recommendation
2. Unified investment thesis
3. Position sizing (% of portfolio)
4. Risk management strategy
5. Key monitoring points
"""
        
        try:
            response = self.model.generate_content(prompt)
            analysis_text = response.text
        except Exception as e:
            analysis_text = f"Synthesis based on voting: Buy={buy_votes:.1f}, Hold={hold_votes:.1f}, Sell={sell_votes:.1f}"
        
        # Determine final recommendation
        if buy_votes > hold_votes and buy_votes > sell_votes:
            final_recommendation = "BUY"
        elif sell_votes > hold_votes and sell_votes > buy_votes:
            final_recommendation = "SELL"
        else:
            final_recommendation = "HOLD"
        
        return AgentResult(
            agent_name="portfolio_manager",
            ticker=context.ticker,
            status=AnalysisStatus.COMPLETED,
            confidence_score=avg_confidence,
            recommendation=final_recommendation,
            key_metrics={
                "buy_votes": buy_votes,
                "hold_votes": hold_votes,
                "sell_votes": sell_votes
            },
            analysis_text=analysis_text,
            risks=["Portfolio concentration risk"],
            opportunities=["Diversification benefit"],
            timestamp=datetime.now().isoformat()
        )
        
print("✅ All agent classes defined.")

✅ All agent classes defined.


## Cell 5: Main Platform (Orchestrator)

The `MultiAgentInvestmentPlatform` class is the orchestrator that manages the entire analysis workflow. It initializes all the agents and defines the sequence of operations:

1.  **`run_comprehensive_analysis`**: Executes the full, three-phase workflow:
    -   **Phase 1**: Data Collection
    -   **Phase 2**: Parallel analysis by Fundamental, Technical, and Sentiment agents.
    -   **Phase 3**: Synthesis by the Portfolio Manager.
2.  **`run_quick_analysis`**: Runs a targeted analysis using only specified agents.
3.  **`generate_analysis_report`**: Formats the final results into a readable report.

In [6]:
class MultiAgentInvestmentPlatform:
    """Main orchestrator managing multiple specialized agents."""
    
    def __init__(self):
        self.tools = SharedInvestmentTools()
        
        # Initialize all agents
        self.data_agent = DataCollectionAgent(self.tools)
        self.fundamental_agent = FundamentalAnalysisAgent(self.tools)
        self.technical_agent = TechnicalAnalysisAgent(self.tools)
        self.sentiment_agent = SentimentAnalysisAgent(self.tools)
        self.portfolio_agent = PortfolioManagerAgent()
        
        print(f"✅ Initialized 5 specialized agents")
    
    async def run_comprehensive_analysis(self, ticker: str) -> Dict[str, AgentResult]:
        """Execute full multi-agent analysis workflow."""
        
        print(f"\n🚀 Starting comprehensive analysis for {ticker}")
        print("=" * 50)
        
        # Initialize context
        context = InvestmentContext(
            ticker=ticker.upper(),
            company_name="",
            sector="",
            market_cap=0,
            current_price=0,
            session_id=f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            analysis_results={}
        )
        
        # PHASE 1: Data Collection
        print("📊 Phase 1: Data Collection...")
        try:
            data_result = await self.data_agent.collect_data(ticker)
            context.add_result(data_result)
            
            # Parse basic info
            basic_info = json.loads(self.tools.get_basic_company_info(ticker))
            context.company_name = basic_info['name']
            context.sector = basic_info['sector']
            context.market_cap = basic_info['market_cap']
            context.current_price = basic_info['current_price']
            
            print(f"✅ Data collection completed")
            
        except Exception as e:
            print(f"❌ Data collection failed: {str(e)}")
            return {"error": f"Data collection failed: {str(e)}"}
        
        # PHASE 2: Parallel Analysis
        print("🔄 Phase 2: Specialist Analysis...")
        
        try:
            # Run analyses in parallel
            fundamental_task = self.fundamental_agent.analyze_fundamentals(context)
            technical_task = self.technical_agent.analyze_technicals(context)
            sentiment_task = self.sentiment_agent.analyze_sentiment(context)
            
            results = await asyncio.gather(
                fundamental_task,
                technical_task,
                sentiment_task,
                return_exceptions=True
            )
            
            for result in results:
                if not isinstance(result, Exception):
                    context.add_result(result)
                    print(f"✅ {result.agent_name}: {result.recommendation}")
                else:
                    print(f"❌ Analysis failed: {str(result)}")
                    
        except Exception as e:
            print(f"❌ Analysis phase failed: {str(e)}")
        
        # PHASE 3: Portfolio Synthesis
        print("🎯 Phase 3: Portfolio Synthesis...")
        try:
            portfolio_result = await self.portfolio_agent.synthesize_analysis(context)
            context.add_result(portfolio_result)
            print(f"✅ Final recommendation: {portfolio_result.recommendation}")
            
        except Exception as e:
            print(f"❌ Portfolio synthesis failed: {str(e)}")
        
        print(f"🏁 Analysis completed for {ticker}")
        return context.analysis_results
    
    async def run_quick_analysis(self, ticker: str, focus_areas: List[str]) -> Dict[str, AgentResult]:
        """Run targeted analysis with specified agents."""
        
        print(f"\n⚡ Quick analysis for {ticker} - Focus: {', '.join(focus_areas)}")
        
        context = InvestmentContext(
            ticker=ticker.upper(),
            company_name="",
            sector="",
            market_cap=0,
            current_price=0,
            session_id=f"quick_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            analysis_results={}
        )
        
        # Always get basic info
        try:
            basic_info = json.loads(self.tools.get_basic_company_info(ticker))
            context.company_name = basic_info['name']
            context.sector = basic_info['sector']
            context.market_cap = basic_info['market_cap']
            context.current_price = basic_info['current_price']
        except:
            pass
        
        # Run requested analyses
        for focus in focus_areas:
            try:
                if focus == "fundamental":
                    result = await self.fundamental_agent.analyze_fundamentals(context)
                    context.add_result(result)
                elif focus == "technical":
                    result = await self.technical_agent.analyze_technicals(context)
                    context.add_result(result)
                elif focus == "sentiment":
                    result = await self.sentiment_agent.analyze_sentiment(context)
                    context.add_result(result)
            except Exception as e:
                print(f"❌ {focus} analysis failed: {str(e)}")
        
        return context.analysis_results
    
    def generate_analysis_report(self, ticker: str, results: Dict[str, AgentResult]) -> str:
        """Generate formatted report."""
        
        if not results:
            return f"No analysis results available for {ticker}"
        
        # Get company info
        try:
            company_info = json.loads(self.tools.get_basic_company_info(ticker))
        except:
            company_info = {"name": ticker, "sector": "Unknown", "market_cap": 0, "current_price": 0}
        
        # Start building the report string with Markdown
        report = f"""# 📈 Investment Analysis Report: {company_info['name']} ({ticker.upper()})\n
---\n
Generated: *{datetime.now().strftime('%Y-%m-%d %H:%M:%S')}*\n\n## 🏢 Company Overview\n- **Sector**: {company_info['sector']}\n- **Market Cap**: ${company_info['market_cap']:,.0f}\n- **Current Price**: ${company_info['current_price']:.2f}\n\n"""
        
        # Add final recommendation at the top if available
        if 'portfolio_manager' in results:
            pm_result = results['portfolio_manager']
            report += f"""##  Final Synthesis: Portfolio Manager Recommendation\n**Final Recommendation: {pm_result.recommendation}** (Confidence: {pm_result.confidence_score:.1%})\n\n> {pm_result.analysis_text.splitlines()[3 if len(pm_result.analysis_text.splitlines()) > 3 else 0]}\n\n---\n"""

        # Add each agent's analysis
        for agent_name, result in results.items():
            if agent_name == 'portfolio_manager' and 'portfolio_manager' in results:
                continue # Skip if we already added the summary
            report += f"""### {agent_name.replace('_', ' ').title()}\n**Recommendation**: {result.recommendation} | **Confidence**: {result.confidence_score:.1%}\n\n{result.analysis_text[:800]}...\n\n---\n"""
        
        return report
    
print("✅ Main platform orchestrator defined.")

✅ Main platform orchestrator defined.


## Cell 6: Command Center

The `InvestmentCommandCenter` provides a simple, text-based interface to interact with the platform. It parses natural language commands to decide which analysis to run.

**Supported Commands:**
- `full analysis [TICKER]`
- `quick technical [TICKER]`
- `quick fundamental [TICKER]`
- `compare [TICKER1] [TICKER2]`

In [7]:
class InvestmentCommandCenter:
    """Interactive interface for the platform."""
    
    def __init__(self):
        self.platform = MultiAgentInvestmentPlatform()
        # For displaying markdown in Jupyter
        from IPython.display import display, Markdown
        self.display = display
        self.Markdown = Markdown
    
    async def process_command(self, command: str):
        """Process investment commands and display the output as markdown."""
        
        command = command.lower().strip()
        
        # Extract tickers
        import re
        potential_tickers = re.findall(r'\b[A-Z]{1,5}\b', command.upper())
        
        report_str = ""
        
        if "full analysis" in command or "comprehensive" in command:
            if potential_tickers:
                ticker = potential_tickers[-1]
                results = await self.platform.run_comprehensive_analysis(ticker)
                report_str = self.platform.generate_analysis_report(ticker, results)
            else:
                report_str = "Please specify a ticker. Example: 'full analysis AAPL'"
        
        elif "quick" in command:
            if potential_tickers:
                ticker = potential_tickers[-1]
                focus_areas = []
                
                if "technical" in command:
                    focus_areas.append("technical")
                if "fundamental" in command:
                    focus_areas.append("fundamental")
                if "sentiment" in command:
                    focus_areas.append("sentiment")
                
                if not focus_areas:
                    focus_areas = ["technical", "fundamental"] # Default quick analysis
                
                results = await self.platform.run_quick_analysis(ticker, focus_areas)
                report_str = self.platform.generate_analysis_report(ticker, results)
            else:
                report_str = "Please specify a ticker. Example: 'quick technical TSLA'"
        
        elif "compare" in command:
            if len(potential_tickers) >= 2:
                comparison = f"# Stock Comparison: {', '.join(potential_tickers)}\n\n"
                for ticker in potential_tickers[:3]: # Limit to 3 for comparison
                    results = await self.platform.run_quick_analysis(ticker, ["fundamental", "technical"])
                    
                    # Create a mini-report for each
                    company_info = json.loads(self.platform.tools.get_basic_company_info(ticker))
                    fund_rec = results.get('fundamental_analyst', AgentResult(status=AnalysisStatus.FAILED, recommendation='N/A', confidence_score=0, agent_name='',ticker='',key_metrics={},analysis_text='',risks=[],opportunities=[],timestamp=''))
                    tech_rec = results.get('technical_analyst', AgentResult(status=AnalysisStatus.FAILED, recommendation='N/A', confidence_score=0, agent_name='',ticker='',key_metrics={},analysis_text='',risks=[],opportunities=[],timestamp=''))
                    
                    comparison += f"## {company_info['name']} ({ticker})\n"
                    comparison += f"- **Fundamental Rec**: {fund_rec.recommendation} (Conf: {fund_rec.confidence_score:.0%})\n"
                    comparison += f"- **Technical Rec**: {tech_rec.recommendation} (Conf: {tech_rec.confidence_score:.0%})\n\n"
                report_str = comparison
            else:
                report_str = "Please specify at least 2 tickers. Example: 'compare AAPL MSFT'"
        
        else:
            report_str = """# 🤖 Investment Command Center
            
Welcome! Please use one of the following commands:
            
- `full analysis [TICKER]` - Runs the complete, multi-agent analysis.
- `quick technical [TICKER]` - Focuses on technical indicators and price action.
- `quick fundamental [TICKER]` - Focuses on financial health and valuation.
- `compare [TICKER1] [TICKER2]` - Provides a high-level comparison of two or more stocks.

**Examples:**
- `full analysis GOOGL`
- `quick technical NVDA`
- `compare TSLA F`
"""
        # Display the output in the notebook
        self.display(self.Markdown(report_str)) 

print("✅ Platform ready to use!")

✅ Platform ready to use!


## Cell 7: Run Analysis

Now it's time to use the platform. 

1.  Instantiate the `InvestmentCommandCenter`.
2.  Use the `await` keyword to call the `process_command` method with your desired command.

Below are some examples. You can edit the commands or add new cells to run your own analyses.

In [8]:
# Initialize the command center
command_center = InvestmentCommandCenter()

✅ Initialized 5 specialized agents


### Example 1: Show Help Menu

In [9]:
# Show the list of available commands
await command_center.process_command("help")

# 🤖 Investment Command Center

Welcome! Please use one of the following commands:

- `full analysis [TICKER]` - Runs the complete, multi-agent analysis.
- `quick technical [TICKER]` - Focuses on technical indicators and price action.
- `quick fundamental [TICKER]` - Focuses on financial health and valuation.
- `compare [TICKER1] [TICKER2]` - Provides a high-level comparison of two or more stocks.

**Examples:**
- `full analysis GOOGL`
- `quick technical NVDA`
- `compare TSLA F`


### Example 2: Full Comprehensive Analysis

This runs the entire workflow, involving all specialist agents and the final portfolio manager.

In [10]:
# Run a full analysis on Google (GOOGL)
# You can change the ticker to any stock you like (e.g., "AAPL", "MSFT", "TSLA")
await command_center.process_command("full analysis AAPL")


🚀 Starting comprehensive analysis for AAPL
📊 Phase 1: Data Collection...
✅ Data collection completed
🔄 Phase 2: Specialist Analysis...
✅ fundamental_analyst: HOLD
✅ technical_analyst: BUY
✅ sentiment_analyst: BUY
🎯 Phase 3: Portfolio Synthesis...
✅ Final recommendation: HOLD
🏁 Analysis completed for AAPL


# 📈 Investment Analysis Report: Apple Inc. (AAPL)

---

Generated: *2025-08-11 01:41:14*

## 🏢 Company Overview
- **Sector**: Technology
- **Market Cap**: $3,403,645,714,432
- **Current Price**: $229.35

##  Final Synthesis: Portfolio Manager Recommendation
**Final Recommendation: HOLD** (Confidence: 73.8%)

> 

---
### Data Collector
**Recommendation**: DATA_COLLECTED | **Confidence**: 85.0%


Data Collection for AAPL:

{
  "name": "Apple Inc.",
  "sector": "Technology",
  "industry": "Consumer Electronics",
  "market_cap": 3403645714432,
  "current_price": 229.35
}

Financial Data for AAPL:
- Revenue: $408,624,988,160
- Net Income: $99,280,003,072
- Profit Margin: 24.30%
- P/E Ratio: 34.855625
- P/B Ratio: 51.760323
- Debt/Equity: 154.486


Market Data for AAPL:
- Current Price: $229.35
- 52-Week High: $259.47
- 52-Week Low: $168.99
- 1-Year Change: +6.56%
- Volume (Avg): 53,536,927
- Beta: 1.165



Okay, let's assess the quality and completeness of the provided AAPL data.

**1. Data Quality Score (0-1):**

I'd give this data a quality score of **0.85**.  Here's the breakdown:

*   **Completeness:**  Generally good, covers key areas (company overview, financial performance, ma...

---
### Fundamental Analyst
**Recommendation**: HOLD | **Confidence**: 75.0%

Analysis error: 404 models/gemini-gemini-2.0-flash is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods....

---
### Technical Analyst
**Recommendation**: BUY | **Confidence**: 70.0%

Okay, here's a technical analysis of AAPL based on the provided information, broken down into the requested sections.

**1. Trend Analysis:**

*   **Overall Trend:** The trend is currently **bullish**. This is supported by the price trading well above both the 20-day and 50-day moving averages. The Moving Averages are also showing a golden cross which further confirms the bullish trend.
*   **Momentum:** The RSI indicates that the stock is approaching overbought levels, but is still considered neutral.

**2. Support and Resistance Levels:**

*   **Resistance:**
    *   **Primary Resistance:** $259.47 (52-Week High).  This is a significant psychological level and where previous buying pressure faltered.
    *   **Secondary Resistance:** $240 - $250 (A zone around the 52 week high less about...

---
### Sentiment Analyst
**Recommendation**: BUY | **Confidence**: 65.0%

Okay, here's an analysis of AAPL based on the provided sentiment data:

**1. Overall Market Sentiment Assessment:**

The market sentiment towards AAPL is **mildly positive**. A sentiment score of 0.37, while positive, isn't overwhelmingly so.  It suggests that the news articles analyzed, on average, leaned towards positive language and viewpoints regarding Apple. However, the score is not high enough to suggest extremely bullish sentiment. We need to consider that this is based on a simulated news environment with only 8 articles.

**2. Impact on Stock Price:**

A mildly positive sentiment generally indicates a **potential for a slight upward pressure** on the stock price.  It suggests investors are likely to feel more confident than apprehensive, which could lead to increased buying.  How...

---


### Example 3: Quick Technical Analysis

This runs a faster, targeted analysis focusing only on technical indicators.

In [11]:
# Run a quick technical analysis on NVIDIA (NVDA)
await command_center.process_command("quick technical NVDA")


⚡ Quick analysis for NVDA - Focus: technical


# 📈 Investment Analysis Report: NVIDIA Corporation (NVDA)

---

Generated: *2025-08-11 01:41:22*

## 🏢 Company Overview
- **Sector**: Technology
- **Market Cap**: $4,456,467,922,944
- **Current Price**: $182.74

### Technical Analyst
**Recommendation**: BUY | **Confidence**: 70.0%

Okay, let's analyze NVDA from a technical perspective based on the data provided.

**1. Trend Analysis:**

*   **Overall Trend:** The 1-Year Change of +74.47% clearly indicates a strong **bullish** trend. The fact that the current price is near the 52-week high ($183.88) further supports this.
*   **Short-Term Trend:** The price trading above both the 20-day and 50-day Moving Averages (MAs) also confirms a short-term uptrend.  The 20-day MA above the 50-day MA is a golden cross formation which further indicates bullish momentum.

**2. Support and Resistance Levels:**

*   **Resistance:**
    *   **Immediate Resistance:** $183.88 (52-Week High).  A breakout above this level could lead to further upside.
    *   **Next Resistance:**  Due to the stock being at its 52 week high, finding concre...

---


### Example 4: Stock Comparison

This command provides a side-by-side summary of fundamental and technical recommendations for multiple stocks.

In [12]:
# Compare Tesla (TSLA) and Ford (F)
await command_center.process_command("compare TSLA F")


⚡ Quick analysis for TSLA - Focus: fundamental, technical

⚡ Quick analysis for F - Focus: fundamental, technical


# Stock Comparison: TSLA, F

## Tesla, Inc. (TSLA)
- **Fundamental Rec**: HOLD (Conf: 75%)
- **Technical Rec**: BUY (Conf: 70%)

## Ford Motor Company (F)
- **Fundamental Rec**: HOLD (Conf: 75%)
- **Technical Rec**: BUY (Conf: 70%)



In [13]:
# From the get_basic_company_info and fetch_financial_statements methods
stock = yf.Ticker("AAPL")
info = stock.info

name = info.get('longName', 'Unknown')
market_cap = info.get('marketCap', 0)
revenue = info.get('totalRevenue', 0)
pe_ratio = info.get('trailingPE', 'N/A')

In [14]:
revenue

408624988160

In [15]:
import google.generativeai as genai
import os

print("🔑 Validating your Google Gemini API key...")

try:
    # Make sure the genai library is configured with your key
    # This line uses the environment variable you've already set
    genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
    
    # This is a simple, free API call to list available models
    # If this works, your key is valid.
    models = [m for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]
    
    # Check if we got any models back
    if models:
        print("✅ Success! Your Gemini API key is valid and working.")
    else:
        print("🟡 Your API key is valid, but no models were found. This is unusual.")

except Exception as e:
    # If the API call fails, it will raise an exception
    print("❌ Failed! Your API key is likely invalid or not configured correctly.")
    print("\n--- Error Details ---")
    print(e)
    print("--------------------")

🔑 Validating your Google Gemini API key...
✅ Success! Your Gemini API key is valid and working.


In [16]:
genai.list_models()

<generator object list_models at 0x1192be020>